# keyness

> Functionality for keyness analysis.

In [ ]:
#| default_exp keyness

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import numpy as np
import time
import polars as pl
from fastcore.basics import patch
import math

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.core import logger, PAGE_SIZE

In [ ]:
#| export
class Keyness:
	""" Class for keyness analysis reporting. """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 
		self.corpus = corpus


In [ ]:
# version from old version of the library
# def keywords(self:Keyness, 
# 			 reference_corpus:Corpus, # Reference corpus
# 			 use_cache:bool=True # retrieve the results from cache if available
# 			 ) -> Result: # Result instance
	
# 	start_time = time.time()

# 	cache_id = tuple(['keywords'])
# 	if use_cache == True and cache_id in self.corpus.results_cache:
# 		keyness_report = self.corpus.results_cache[cache_id]
# 	else:

# 		minimum_reference_normalised_freq = math.log2(0.5/reference_corpus['token_count'])

# 		normalised_freq_in_corpus = np.array(list(loaded_corpora[corpus_name]['frequency_lookup'].values()))/loaded_corpora[corpus_name]['token_count']
# 		normalised_freq_in_reference = np.array(list(reference_corpus['frequency_lookup'].values()))/reference_corpus['token_count']

# 		normalised_freq_in_corpus = np.log2(normalised_freq_in_corpus)
# 		normalised_freq_in_reference = np.log2(normalised_freq_in_reference)

# 		normalised_freq_in_corpus = dict(zip(loaded_corpora[corpus_name]['frequency_lookup'].keys(), normalised_freq_in_corpus.tolist()))
# 		normalised_freq_in_reference = dict(zip(reference_corpus['frequency_lookup'].keys(), normalised_freq_in_reference.tolist()))

# 		keyness_data = {}
# 		for token_id in normalised_freq_in_corpus:
# 			if token_id in normalised_freq_in_reference:
# 				log_ratio = normalised_freq_in_corpus[token_id] - normalised_freq_in_reference[token_id]
# 			else:
# 				log_ratio = normalised_freq_in_corpus[token_id] - minimum_reference_normalised_freq
# 			keyness_data[token_id] = {'id': loaded_corpora[corpus_name]['vocab'][token_id], 'log_ratio': log_ratio}

# 		keyness_report = pd.DataFrame.from_dict(keyness_data, orient='index').sort_values(['log_ratio'], ascending=False)

# 		self.corpus.results_cache[cache_id] = keyness_report

# 	return keyness_report

In [ ]:
#| hide
path_to_brown_corpus = '../test-corpora/saved/brown.corpus'
path_to_reuters_corpus = '../test-corpora/saved/reuters.corpus'
path_to_gutenberg_corpus = '../test-corpora/saved/gutenberg.corpus'


In [ ]:
# load the corpus
brown = Corpus().load(path_to_brown_corpus)
reuters = Corpus().load(path_to_reuters_corpus)
gutenberg = Corpus().load(path_to_gutenberg_corpus)


In [ ]:
brown.summary()
reuters.summary()
gutenberg.summary()


Corpus Summary 
 
 
 
 
 
 Attribute 
 Value 
 
 
 
 
 Name 
 Brown Corpus 
 
 
 Description 
 A Standard Corpus of Present-Day Edited American English, for use with Digital Computers. by W. N. Francis and H. Kucera (1964) Department of Linguistics, Brown University Providence, Rhode Island, USA Revised 1971, Revised and Amplified 1979 http://www.hit.uib.no/icame/brown/bcm.html 
 
 
 Conc Version 
 0.0.1 
 
 
 Corpus Path 
 ../test-corpora/saved/brown.corpus 
 
 
 Source Path 
 ../test-corpora/source//brown.csv.gz 
 
 
 Document Count 
 500 
 
 
 Token Count 
 1,140,905 
 
 
 Unique Tokens 
 42,937 
 
 
 Word Token Count 
 980,144 
 
 
 Unique Word Tokens 
 42,907

Corpus Summary 
 
 
 
 
 
 Attribute 
 Value 
 
 
 
 
 Name 
 Reuters Corpus 
 
 
 Description 
 From NLTK TODO 
 
 
 Conc Version 
 0.0.1 
 
 
 Corpus Path 
 ../test-corpora/saved/reuters.corpus 
 
 
 Source Path 
 ../test-corpora/source/reuters.csv.gz 
 
 
 Document Count 
 10,788 
 
 
 Token Count 
 1,726,826 
 
 
 Unique Tokens 
 50,047 
 
 
 Word Token Count 
 1,398,782 
 
 
 Unique Word Tokens 
 49,860

Corpus Summary 
 
 
 
 
 
 Attribute 
 Value 
 
 
 
 
 Name 
 Gutenberg Corpus 
 
 
 Description 
 From NLTK TODO 
 
 
 Conc Version 
 0.0.1 
 
 
 Corpus Path 
 ../test-corpora/saved/gutenberg.corpus 
 
 
 Source Path 
 ../test-corpora/source/gutenberg.csv.gz 
 
 
 Document Count 
 18 
 
 
 Token Count 
 2,777,046 
 
 
 Unique Tokens 
 51,590 
 
 
 Word Token Count 
 2,162,293 
 
 
 Unique Word Tokens 
 51,393

In [ ]:
from conc.frequency import Frequency

In [ ]:
frequencies_brown = Frequency(brown)
frequencies_reuters = Frequency(reuters)
frequencies_gutenberg = Frequency(gutenberg)

frequencies_brown.frequencies(normalize_by=1000000).display()
frequencies_reuters.frequencies(normalize_by=1000000).display()
frequencies_gutenberg.frequencies(normalize_by=1000000).display()


Frequencies 
 
 
 Frequencies of tokens in the corpus 
 
 
 Rank 
 Token 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the 
 63516 
 64,802.72 
 
 
 2 
 of 
 36321 
 37,056.80 
 
 
 3 
 and 
 27787 
 28,349.92 
 
 
 4 
 to 
 25868 
 26,392.04 
 
 
 5 
 a 
 22190 
 22,639.53 
 
 
 6 
 in 
 19751 
 20,151.12 
 
 
 7 
 that 
 10409 
 10,619.87 
 
 
 8 
 is 
 10138 
 10,343.38 
 
 
 9 
 was 
 9931 
 10,132.18 
 
 
 10 
 for 
 8905 
 9,085.40 
 
 
 11 
 with 
 7043 
 7,185.68 
 
 
 12 
 it 
 6991 
 7,132.63 
 
 
 13 
 he 
 6772 
 6,909.19 
 
 
 14 
 as 
 6738 
 6,874.50 
 
 
 15 
 his 
 6523 
 6,655.14 
 
 
 16 
 on 
 6459 
 6,589.85 
 
 
 17 
 be 
 6365 
 6,493.94 
 
 
 18 
 's 
 5285 
 5,392.06 
 
 
 19 
 had 
 5200 
 5,305.34 
 
 
 20 
 by 
 5156 
 5,260.45 
 
 
 
 
 
 Report based on word tokens 
 


 
 Normalized Frequency is per 1,000,000 tokens 
 


 
 Total tokens: 980,144 
 


 
 Unique tokens: 42,907 
 


 
 Showing 20 rows 
 


 
 Page 1 of 49008

Frequencies 
 
 
 Frequencies of tokens in the corpus 
 
 
 Rank 
 Token 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the 
 69263 
 49,516.65 
 
 
 2 
 of 
 36779 
 26,293.59 
 
 
 3 
 to 
 36328 
 25,971.17 
 
 
 4 
 in 
 29252 
 20,912.48 
 
 
 5 
 and 
 25645 
 18,333.81 
 
 
 6 
 said 
 25379 
 18,143.64 
 
 
 7 
 a 
 24844 
 17,761.17 
 
 
 8 
 mln 
 18621 
 13,312.30 
 
 
 9 
 vs 
 14332 
 10,246.06 
 
 
 10 
 for 
 13720 
 9,808.53 
 
 
 11 
 dlrs 
 12411 
 8,872.72 
 
 
 12 
 it 
 11104 
 7,938.33 
 
 
 13 
 pct 
 9810 
 7,013.24 
 
 
 14 
 's 
 9627 
 6,882.42 
 
 
 15 
 on 
 9244 
 6,608.61 
 
 
 16 
 cts 
 8357 
 5,974.48 
 
 
 17 
 from 
 8216 
 5,873.68 
 
 
 18 
 is 
 7673 
 5,485.49 
 
 
 19 
 that 
 7540 
 5,390.40 
 
 
 20 
 year 
 7523 
 5,378.25 
 
 
 
 
 
 Report based on word tokens 
 


 
 Normalized Frequency is per 1,000,000 tokens 
 


 
 Total tokens: 1,398,782 
 


 
 Unique tokens: 49,860 
 


 
 Showing 20 rows 
 


 
 Page 1 of 69940

Frequencies 
 
 
 Frequencies of tokens in the corpus 
 
 
 Rank 
 Token 
 Frequency 
 Normalized Frequency 
 
 
 
 
 1 
 the 
 133482 
 61,731.69 
 
 
 2 
 and 
 95243 
 44,047.22 
 
 
 3 
 of 
 71241 
 32,946.97 
 
 
 4 
 to 
 48003 
 22,200.04 
 
 
 5 
 a 
 33854 
 15,656.53 
 
 
 6 
 in 
 33530 
 15,506.69 
 
 
 7 
 i 
 29872 
 13,814.96 
 
 
 8 
 that 
 28728 
 13,285.90 
 
 
 9 
 he 
 25774 
 11,919.75 
 
 
 10 
 it 
 22107 
 10,223.87 
 
 
 11 
 his 
 21391 
 9,892.74 
 
 
 12 
 for 
 19474 
 9,006.18 
 
 
 13 
 was 
 18730 
 8,662.10 
 
 
 14 
 not 
 18164 
 8,400.34 
 
 
 15 
 with 
 17573 
 8,127.02 
 
 
 16 
 is 
 16433 
 7,599.80 
 
 
 17 
 you 
 16288 
 7,532.74 
 
 
 18 
 be 
 16094 
 7,443.02 
 
 
 19 
 as 
 14510 
 6,710.47 
 
 
 20 
 but 
 13773 
 6,369.63 
 
 
 
 
 
 Report based on word tokens 
 


 
 Normalized Frequency is per 1,000,000 tokens 
 


 
 Total tokens: 2,162,293 
 


 
 Unique tokens: 51,393 
 


 
 Showing 20 rows 
 


 
 Page 1 of 108115

In [ ]:
from conc.concordance import Concordance

In [ ]:
brown_concordance = Concordance(brown)
reuters_concordance = Concordance(reuters)
gutenberg_concordance = Concordance(gutenberg)

In [ ]:
import string
import re

In [ ]:
reference = brown
target = gutenberg

reference_df = reference.frequency_table.sort('frequency', descending=True).filter(pl.col('is_punct') == False).filter(pl.col('is_space') == False)
target_df = target.frequency_table.sort('frequency', descending=True).filter(pl.col('is_punct') == False).filter(pl.col('is_space') == False)
# create new pl df joined by token
reference_min_freq = 0.05 * 1000000 / reference.word_token_count
target_min_freq = 0.05 * 1000000 / target.word_token_count
combined_frequency_table = target_df.join(reference_df, on='token', how='left', suffix = '_reference').drop('rank', 'token_id', 'is_punct', 'is_space', 'rank_reference', 'token_id_reference', 'is_punct_reference', 'is_space_reference')

# polars df -  replace null values in normalized_frequency_reference with the brown_min_frequency
combined_frequency_table = combined_frequency_table.with_columns(pl.col('normalized_frequency_reference').fill_null(reference_min_freq))
combined_frequency_table = combined_frequency_table.with_columns((pl.col('normalized_frequency')/pl.col('normalized_frequency_reference')).alias('relative_risk'))

print(combined_frequency_table.filter(pl.col('frequency') > 100).sort('relative_risk', descending=True).head(20))


┌───────────┬───────────┬───────────────────┬───────────────────┬──────────────────┬───────────────┐
│ frequency ┆ token     ┆ normalized_freque ┆ frequency_referen ┆ normalized_frequ ┆ relative_risk │
│           ┆           ┆ ncy               ┆ ce                ┆ ency_reference   ┆               │
╞═══════════╪═══════════╪═══════════════════╪═══════════════════╪══════════════════╪═══════════════╡
│ 1673      ┆ shalt     ┆ 773.715681        ┆ null              ┆ 0.051013         ┆ 15167.05564   │
│ 1246      ┆ whale     ┆ 576.24013         ┆ null              ┆ 0.051013         ┆ 11295.966125  │
│ 817       ┆ judah     ┆ 377.839636        ┆ null              ┆ 0.051013         ┆ 7406.745043   │
│ 652       ┆ spake     ┆ 301.531754        ┆ null              ┆ 0.051013         ┆ 5910.890781   │
│ 627       ┆ marianne  ┆ 289.969953        ┆ null              ┆ 0.051013         ┆ 5684.246196   │
│ 591       ┆ ahab      ┆ 273.320961        ┆ null              ┆ 0.051013         ┆ 5357.8

In [ ]:
# instantiate the Ngrams class
report_brown = Keyness(brown)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()